In [2]:
import numpy as np
import torch
import torch.nn as nn

In [12]:
def softmax(Z):
    Z = np.exp(Z - Z.max(axis=-1, keepdims=True))
    return Z / Z.sum(axis=-1, keepdims=True)

def self_attention(X, mask, W_KQV, W_out):
    K, Q, V = np.split(X @ W_KQV, 3, axis=1)
    attn = softmax(K @ Q.T / np.sqrt(X.shape[1]) + mask)
    return attn @ V @ W_out, attn

In [4]:
T, d = 100, 64
attn = nn.MultiheadAttention(d, 1, bias=False, batch_first=True)
M = torch.triu(-float('inf') * torch.ones(T, T), 1)
X = torch.randn(1, T, d)
Y_, A_ = attn(X, X, X, attn_mask=M)

In [13]:
Y, A = self_attention(X[0].numpy(), M.numpy(), attn.in_proj_weight.detach().numpy().T, attn.out_proj.weight.detach().numpy().T)

In [14]:
np.linalg.norm(Y - Y_.detach().numpy())

1.18326e-06

# Minibatch

In [15]:
C = np.random.randn(5, 4, 10, 3)
D = np.random.randn(3, 6)
(C @ D).shape

(5, 4, 10, 6)

In [22]:
def self_attention(X, mask, W_KQV, W_out):
    K, Q, V = np.split(X @ W_KQV, 3, axis=-1)
    attn = softmax(K @ Q.swapaxes(-1, -2) / np.sqrt(X.shape[-1]) + mask)
    return attn @ V @ W_out, attn

In [17]:
B, T, d = 50, 100, 64
X = torch.randn(B, T, d)
M = torch.triu(-float('inf') * torch.ones(T, T), 1)
Y_, A_ = attn(X, X, X, attn_mask=M)

In [23]:
Y, A = self_attention(X.numpy(), M.numpy(), attn.in_proj_weight.detach().numpy().T, attn.out_proj.weight.detach().numpy().T)

In [24]:
np.linalg.norm(Y - Y_.detach().numpy())

8.6435775e-06